In [ ]:
import pandas as pd
import os
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import login
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments,EarlyStoppingCallback
from trl import SFTTrainer, SFTConfig
import torch
import re
from tqdm import tqdm
import gc
from sklearn.metrics import f1_score
from dataclasses import dataclass

In [ ]:
!pip install gradio accelerate datasets bitsandbytes peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
login("hf_token")

In [ ]:
!nvidia-smi

Tue Jul  1 17:06:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             48W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
dataset = load_dataset("openai/gsm8k", "main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
final_answer_prompt = (
    "Solve the following math problem step by step. Perform all necessary calculations carefully. At the end, output only the final answer. Provide a stepe-by-step reasoning."
)

In [ ]:
dataset_combined = concatenate_datasets([dataset['train'], dataset['test']])

In [ ]:
import re

def clean_text(text):
    if not text:
        return text

    text = text.replace('\xa0', ' ')
    text = text.replace('\u200b', '')
    text = text.replace('\u2028', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('‘', "'").replace('’', "'")
    text = text.replace('“', '"').replace('”', '"')
    text = text.replace('–', '-').replace('—', '-')
    text = text.replace('−', '-')
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [ ]:
def clean_dataset_row(row):
    row['answer'] = clean_text(row['answer'])
    row['question'] = clean_text(row['question'])
    return row

dataset_combined = dataset_combined.map(clean_dataset_row)
dataset_combined = dataset_combined.to_pandas()


Map:   0%|          | 0/8792 [00:00<?, ? examples/s]

In [ ]:
def load_model_and_tokenizer(model_name):
  bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

  model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token


  return model, tokenizer

model, tokenizer = load_model_and_tokenizer("microsoft/phi-4")

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

In [ ]:
import re

def extract_final_number(text):
    match = re.search(r"####\s*([-+]?\d*\.?\d+)", text)
    if match:
        return match.group(1).strip()

    all_numbers = re.findall(r"[-+]?\d*\.?\d+", text)
    return all_numbers[-1].strip() if all_numbers else ""

In [ ]:
def format_questions_answers(data_samples, llm_prompt):
    EOS_TOKEN = tokenizer.eos_token
    questions_column = data_samples['question']
    answers_column = data_samples['answer']

    formatted_texts = []

    for question, answer in zip(questions_column, answers_column):
        prompt = f"[INST] {llm_prompt.strip()}\n{question.strip()} [/INST]\n{answer.strip()}\n#### {extract_final_number(answer)}{EOS_TOKEN}"
        formatted_texts.append(prompt)

    return {'text': formatted_texts}

In [ ]:
dataset_train = dataset['train']
dataset_test = dataset['test']

dataset_train_updated = dataset_train.map(lambda x: format_questions_answers(x, final_answer_prompt), batched = True)
dataset_test_updated = dataset_test.map(lambda x: format_questions_answers(x, final_answer_prompt), batched = True)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
dataset_train_updated = dataset_train_updated.shuffle(seed = 42)

In [ ]:
peft_config = LoraConfig(
                          lora_alpha=32,
                          lora_dropout=0.1,
                          r=32,
                          bias="lora_only",
                          task_type="CAUSAL_LM",
                          target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
  )

In [ ]:
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = dataset_train_updated,
    eval_dataset = dataset_test_updated,
    peft_config = peft_config,
    compute_metrics=compute_metrics,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 2200,
        max_seq_length = 1024,
        learning_rate = 2e-5,
        fp16 = False,
        bf16 = True,
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
    ),
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Adding EOS to train dataset:   0%|          | 0/7473 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7473 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/7473 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/1319 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1319 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1319 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
100,0.626300
200,0.322300
300,0.314000
400,0.298300
500,0.304400
600,0.304100
700,0.301700
800,0.292300
900,0.301800
1000,0.294700


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
100,0.626300
200,0.322300
300,0.314000
400,0.298300
500,0.304400
600,0.304100
700,0.301700
800,0.292300
900,0.301800
1000,0.294700


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=2200, training_loss=0.3085488796234131, metrics={'train_runtime': 8049.2515, 'train_samples_per_second': 2.187, 'train_steps_per_second': 0.273, 'total_flos': 3.019411702977331e+17, 'train_loss': 0.3085488796234131})

In [ ]:
def cleanup():

    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
cleanup()

In [ ]:
def compute_llm_eval_answers(eval_set, chat_instruction, model_ftuned, tokenizer_ftuned):
    llm_clean_answers = []
    few_shot_examples = """
Q: Tom has 3 toy cars. He receives 2 more from his friend. How many toy cars does he have now?
A: We add because he is getting more.
3 + 2 = 5
The '+' symbol means we are combining amounts.
#### 5

Q: Sarah had 10 cookies. She gave 4 cookies to her friend. How many cookies does she have left?
A: We subtract because she is giving some away.
10 - 4 = 6
The '-' symbol means we are removing or taking away.
#### 6

Q: There are 6 packs of markers. Each pack contains 4 markers. How many markers are there in total?
A: We multiply because we have equal groups.
6 × 4 = 24
The '×' symbol means repeated addition.
#### 24

Q: A teacher has 20 pencils and wants to give them equally to 5 students. How many pencils does each student get?
A: We divide because the pencils are split evenly.
20 ÷ 5 = 4
The '÷' symbol means splitting into equal parts.
#### 4

Q: Emma buys 3 notebooks. Each notebook costs 4 dollars. She also buys a pen that costs 2 dollars. How much does she spend in total?
A: First, we calculate the cost of the notebooks:
3 × 4 = 12
Then we add the cost of the pen:
12 + 2 = 14
She spends 14 dollars in total.
#### 14

Q: Mia buys a painting for $200. She spends $50 on restoring it. Later, its value increases by 100%. How much profit did she make?
A: First, we calculate the total investment:
200 + 50 = 250
Then we calculate the new value after a 100% increase:
100% of 250 is 250, so the painting is now worth:
250 + 250 = 500
Now we find the profit:
500 - 250 = 250
She made a profit of $250.
#### 250

Q: Lily ran 3 miles each day for 5 days. Then she ran 2 more miles on the sixth day. How many miles did she run in total?
A: First, calculate the distance she ran in 5 days:
3 × 5 = 15
Then add the 2 miles from the sixth day:
15 + 2 = 17
She ran a total of 17 miles.
#### 17

Q: A pizza is cut into 8 slices. Emma eats 3 slices, and Noah eats 2 slices. What fraction of the pizza is left?
A: First, we find the total slices eaten:
3 + 2 = 5
Then subtract from the total:
8 - 5 = 3
3 out of 8 slices are left, so the fraction is:
3/8
#### 3/8
"""

    model_ftuned.eval()
    with torch.no_grad():
        for _, problem_instruction in enumerate(tqdm(eval_set)):
            prompt = (
    f"{chat_instruction.strip()}\n\n"
    f"{few_shot_examples.strip()}\n\n"
    f"Q: {problem_instruction.strip()}\nA:"
)

            input_ids = tokenizer_ftuned.encode(prompt, return_tensors="pt").to(model_ftuned.device)
            input_ids = input_ids.to(dtype=torch.long)

            with torch.autocast("cuda", dtype=torch.bfloat16):
                outputs = model_ftuned.generate(
                    input_ids=input_ids,
                    max_new_tokens=512,
                    do_sample=False,
                    temperature=0.0
                )

            out = tokenizer_ftuned.decode(outputs[0], skip_special_tokens=False)

            clean_answer = out.replace('</s>', '').strip()
            if '####' in clean_answer:
                clean_answer = clean_answer.split('####')[-1].strip()

            llm_clean_answers.append(clean_answer)

    return llm_clean_answers.

llm_clean= compute_llm_eval_answers(dataset_test_updated, final_answer_prompt, model, tokenizer)

100%|██████████| 3/3 [01:07<00:00, 22.62s/it]


In [ ]:
pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
from bert_score import score

ground_truth_cleaned = []
for ans in dataset_test_updated['text']:
    parts = ans.split('[/INST]')
    if len(parts) > 1:
        cleaned_ans = parts[1].replace('</s>', '').strip()
    else:
        cleaned_ans = ans.replace('</s>', '').strip()

    ground_truth_cleaned.append(cleaned_ans)

min_len_bert = min(len(llm_clean), len(ground_truth_cleaned))
P, R, F1 = score(llm_clean[:min_len_bert], ground_truth_cleaned[:min_len_bert], lang="en", verbose=True)

print(f"Mean BERTScore F1: {F1.mean().item():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.16 seconds, 18.62 sentences/sec
Mean BERTScore F1: 0.5584


In [ ]:
import re
import math
from fractions import Fraction

def extract_answer_after_hashes(text):
    """Extrage numărul după #### dintr-un răspuns de referință"""
    match = re.search(r"####\s*([-+]?[\d\s./]+)", text)
    return match.group(1).strip() if match else None

def safe_to_float(value):
    """Transformă valoarea (număr, zecimal sau fracție) în float"""
    try:
        return float(Fraction(value))
    except:
        return None

def clean_llm_output(text):
    """Curăță textul generat de LLM (ex. elimină <|end_of_text|> și spații)"""
    if text:
        return text.replace("<|end_of_text|>", "").strip()
    return ""

ground_truth_final_values = [extract_answer_after_hashes(text) for text in ground_truth_cleaned]


results_dict = {}
for idx, (pred, true_val) in enumerate(zip(llm_clean, ground_truth_final_values)):
    pred_clean = clean_llm_output(pred)
    pred_clean = re.sub(r"[^\d./\- ]", "", pred_clean).strip()  # 👈 adăugat

    pred_float = safe_to_float(pred_clean)
    true_float = safe_to_float(true_val)

    exact_match = pred_float == true_float
    tolerant_match = (
        math.isclose(pred_float, true_float, rel_tol=1e-2)
        if pred_float is not None and true_float is not None
        else False
    )

    results_dict[idx] = {
        "llm_answer_raw": pred,
        "true_answer_raw": true_val,
        "llm_answer": pred_float,
        "true_answer": true_float,
        "exact_match": exact_match,
        "tolerant_match": tolerant_match
    }

total = len(results_dict)
exact_matches = sum(x["exact_match"] for x in results_dict.values())
tolerant_matches = sum(x["tolerant_match"] for x in results_dict.values())

exact_accuracy = exact_matches / total
tolerant_accuracy = tolerant_matches / total

print(f"Total: {total}")
print(f"Exact matches: {exact_matches} → Accuracy: {exact_accuracy:.2%}")
print(f"Tolerant matches (±1%): {tolerant_matches} → Accuracy: {tolerant_accuracy:.2%}")

for k in range(total):
    print(f"Q{k}: {results_dict[k]}")

Total: 3
Exact matches: 2 → Accuracy: 66.67%
Tolerant matches (±1%): 2 → Accuracy: 66.67%
Q0: {'llm_answer_raw': '18<|im_end|>', 'true_answer_raw': '18', 'llm_answer': 18.0, 'true_answer': 18.0, 'exact_match': True, 'tolerant_match': True}
Q1: {'llm_answer_raw': '', 'true_answer_raw': '3', 'llm_answer': None, 'true_answer': 3.0, 'exact_match': False, 'tolerant_match': False}
Q2: {'llm_answer_raw': '70000<|im_end|>', 'true_answer_raw': '70000', 'llm_answer': 70000.0, 'true_answer': 70000.0, 'exact_match': True, 'tolerant_match': True}


In [ ]:
import gradio as gr
import torch

def compute_llm_eval_single(question, chat_instruction, model_ftuned, tokenizer_ftuned):
    few_shot_examples = """
Q: Tom has 3 toy cars. He receives 2 more from his friend. How many toy cars does he have now?
A: We add because he is getting more.
3 + 2 = 5
The '+' symbol means we are combining amounts.
#### 5

Q: Sarah had 10 cookies. She gave 4 cookies to her friend. How many cookies does she have left?
A: We subtract because she is giving some away.
10 - 4 = 6
The '-' symbol means we are removing or taking away.
#### 6

Q: There are 6 packs of markers. Each pack contains 4 markers. How many markers are there in total?
A: We multiply because we have equal groups.
6 × 4 = 24
The '×' symbol means repeated addition.
#### 24

Q: A teacher has 20 pencils and wants to give them equally to 5 students. How many pencils does each student get?
A: We divide because the pencils are split evenly.
20 ÷ 5 = 4
The '÷' symbol means splitting into equal parts.
#### 4

Q: Emma buys 3 notebooks. Each notebook costs 4 dollars. She also buys a pen that costs 2 dollars. How much does she spend in total?
A: First, we calculate the cost of the notebooks:
3 × 4 = 12
Then we add the cost of the pen:
12 + 2 = 14
She spends 14 dollars in total.
#### 14

Q: Mia buys a painting for $200. She spends $50 on restoring it. Later, its value increases by 100%. How much profit did she make?
A: First, we calculate the total investment:
200 + 50 = 250
Then we calculate the new value after a 100% increase:
100% of 250 is 250, so the painting is now worth:
250 + 250 = 500
Now we find the profit:
500 - 250 = 250
She made a profit of $250.
#### 250

Q: Lily ran 3 miles each day for 5 days. Then she ran 2 more miles on the sixth day. How many miles did she run in total?
A: First, calculate the distance she ran in 5 days:
3 × 5 = 15
Then add the 2 miles from the sixth day:
15 + 2 = 17
She ran a total of 17 miles.
#### 17

Q: A pizza is cut into 8 slices. Emma eats 3 slices, and Noah eats 2 slices. What fraction of the pizza is left?
A: First, we find the total slices eaten:
3 + 2 = 5
Then subtract from the total:
8 - 5 = 3
3 out of 8 slices are left, so the fraction is:
3/8
#### 3/8
"""

    prompt = (
        f"{chat_instruction.strip()}\n\n"
        f"{few_shot_examples.strip()}\n\n"
        f"Q: {question.strip()}\nA:"
    )

    model_ftuned.eval()
    with torch.no_grad():
        inputs = tokenizer_ftuned(prompt, return_tensors="pt").to(model_ftuned.device)

        with torch.autocast("cuda", dtype=torch.bfloat16):
            outputs = model_ftuned.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                temperature=0.0,
                num_beams=1
            )

        full_output = tokenizer_ftuned.decode(outputs[0], skip_special_tokens=False)

        if f"Q: {question.strip()}" in full_output:
            extracted = full_output.split(f"Q: {question.strip()}")[-1].strip()

            extracted = extracted.split("####")[0].strip()
            extracted = extracted.replace("<|im_end|>", "").strip()

            return f"Q: {question.strip()}\n{extracted}"
        else:
            return full_output.strip()

def gradio_interface(question):
    return compute_llm_eval_single(question, final_answer_prompt, model, tokenizer)

gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=4, label="Math Problem"),
    outputs=gr.Textbox(label="Model Answer"),
    title="Math Solver (LLM)",
    description="Enter a math problem and get a step-by-step answer with the final result.",
    allow_flagging="never"
).launch()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://18ca5a904e6f5f114b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
